In [1]:
import pandas as pd
import numpy as np
import pyodbc
import pycountry

In [2]:
# loading data
df = pd.read_csv('data/preprocessed_data.csv')
df.columns

Index(['Company', 'Job Title', 'Location', 'Job Type', 'Experience level',
       'Salary', 'Requirment of the company ', 'Facilities', 'country',
       'Salary_in_1000_USD', 'Negociable', 'Asterisk', 'Job_category'],
      dtype='object')

## Connecting to the database
---

In [3]:
driver = 'SQL Server'
server = 'LAPTOP-K8C2EPLP\SQLEXPRESS'
database = 'data_science_jobs'


connection_string = f"""
                            DRIVER={driver};
                            SERVER={server};
                            DATABASE={database};
                            Trusted_Connection=yes;
"""
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()
cursor

In [4]:
# check if the tables were already created
tables = cursor.tables(tableType='TABLE', schema='dbo').fetchall()
tables_names = [table.table_name for table in tables if table.table_schem == 'dbo']
tables_names

[]

In [5]:
# execute the .sql file
if not tables_names :
    cursor.execute(open('sql_server_db/creation.sql').read()[3:])
    connection.commit()

In [6]:
tables = cursor.tables(tableType='TABLE', schema='dbo').fetchall()
tables_names = [table.table_name for table in tables if table.table_schem == 'dbo']
tables_names

['Company',
 'Country',
 'Facilities',
 'Facilities_of_the_job',
 'Job_posts',
 'Job_Titles',
 'Job_Titles_of_the_job',
 'Location',
 'Requirment_of_the_job',
 'Requirments']

In [7]:
# check columns for each table
for table_name in tables_names:
    columns = cursor.columns(table=table_name, schema='dbo').fetchall()
    print(f'Columns for {table_name}:')
    for column in columns:
        print(column.column_name)
    print('\n')

Columns for Company:
Id
Name


Columns for Country:
Country_code
Country_name


Columns for Facilities:
Id
Name


Columns for Facilities_of_the_job:
Job_id
Facilitie_id


Columns for Job_posts:
Id
Company_id
Location_id
Job_Type
Experience_level
Salary_in_1000_USD
Negotiable
Asterisk
Job_category


Columns for Job_Titles:
Id
Name


Columns for Job_Titles_of_the_job:
Job_id
Job_Post_id


Columns for Location:
Id
Full_location
Country_code


Columns for Requirment_of_the_job:
Job_id
Requirment_of_the_company_id


Columns for Requirments:
Id
Name




## Incerting to the data
---

In [8]:
# Function to insert data into a table
def insert_data(table_name, data):
    columns = cursor.columns(table=table_name, schema='dbo').fetchall()
    columns_names = [column.column_name for column in columns]
    columns_names = ', '.join(columns_names)
    values = ', '.join(['?' for _ in range(len(columns_names.split(', ')))])
    query = f"INSERT INTO {table_name} ({columns_names}) VALUES ({values})"
    cursor.executemany(query, data)
    connection.commit()


# company -> facilities -> requierements -> country -> location -> job_posts -> facilities_of_job -> requierements of job -> Job_Titles -> Job_Titles_Of_Job

## Company
---

In [9]:
# fill company table
companies_df = pd.DataFrame({'Company': df['Company'].unique()})
# generate an id for each company 
companies_df['Company_id'] = companies_df['Company'].astype('category').cat.codes
companies_df = companies_df[['Company_id', 'Company']]
# as a list of tuples
companies = list(companies_df.itertuples(index=False, name=None))
# insert data into the table
insert_data('Company', companies)

In [10]:
cursor.execute('SELECT * FROM Company').fetchmany(10)

[(0, '15Five'),
 (1, '23andMe'),
 (2, '24-7 Intouch'),
 (3, '2K'),
 (4, '2U'),
 (5, '360dialog GmbH'),
 (6, '3Cloud'),
 (7, '3Pillar Global'),
 (8, '605'),
 (9, '66degrees')]

## Facilities
---

In [11]:
# group Requirment of the company in 1 list then get the unique values
df['Facilities'] = df['Facilities'].apply(lambda x: x.split(','))
df['Facilities'] = df['Facilities'].apply(lambda x: list(set(x)))

# get the unique values of the list
facilities = list(set([j for i in df['Facilities'].values for j in i]))
# generate an id for each facility
facilities_df = pd.DataFrame({'Facility': facilities})
facilities_df['Facility_id'] = facilities_df['Facility'].astype('category').cat.codes
facilities_df = facilities_df[['Facility_id', 'Facility']]
facilities_df = facilities_df[1:]
facilities_df = [tuple(row) for row in facilities_df.values.tolist()]
# insert data into the table
insert_data('Facilities', facilities_df)
cursor.execute('SELECT * FROM Facilities').fetchmany(10)

[(1, '401(k) matching'),
 (2, 'Career development'),
 (3, 'Cell phone stipend'),
 (4, 'Competitive pay'),
 (5, 'Conferences'),
 (6, 'Contract'),
 (7, 'Equity'),
 (8, 'Fertility benefits'),
 (9, 'Fitness / gym'),
 (10, 'Flat hierarchy')]

## Requierements
---

In [12]:
# group Requirment of the company in 1 list then get the unique values
df['Requirment of the company '] = df['Requirment of the company '].apply(lambda x: x.split(','))
df['Requirment of the company '] = df['Requirment of the company '].apply(lambda x: list(set(x)))

# get the unique values of the list
job_requirements = list(set([j for i in df['Requirment of the company '].values for j in i]))
# generate an id for each job requirement
job_requirements_df = pd.DataFrame({'Job_requirement': job_requirements})
job_requirements_df['Job_requirement_id'] = job_requirements_df['Job_requirement'].astype('category').cat.codes
job_requirements_df = job_requirements_df[['Job_requirement_id', 'Job_requirement']]
job_requirements_df = job_requirements_df[1:]
job_requirements_df = [tuple(row) for row in job_requirements_df.values.tolist()]
# insert data into the table
insert_data('Requirments', job_requirements_df)
cursor.execute('SELECT * FROM Requirments').fetchmany(10)

[(1, '.NET'),
 (2, '3D Reconstruction'),
 (3, '3D graphics'),
 (4, 'A/B testing'),
 (5, 'AGI'),
 (6, 'AI art'),
 (7, 'AI content'),
 (8, 'AI governance'),
 (9, 'AI strategy'),
 (10, 'AIStats')]

## Country
---

In [13]:
# get the unique values of countries
countries = pd.DataFrame({'country': df['country'].unique()})
# get country codes using geopy if its not 'Remote' or 'remote'
countries_names = [country.name for country in pycountry.countries]
countries_names.append('Remote')
countries_codes = [country.alpha_2 for country in pycountry.countries]
countries_codes.append('Remote')
# fill the dataframe with country codes
countries_codes = pd.DataFrame({'country': countries_names, 'country_code': countries_codes}, index=None)
# merge the 2 dataframes
countries = countries.merge(countries_codes, on='country', how='left')
countries = countries[['country_code', 'country']]
#as a list of tuples
countries = [tuple(row) for row in countries.values.tolist()]
# insert data into the table
insert_data('Country', countries)
cursor.execute('SELECT * FROM Country').fetchmany(10)

[('AD', 'Andorra'),
 ('AE', 'United Arab Emirates'),
 ('AF', 'Afghanistan'),
 ('AG', 'Antigua and Barbuda'),
 ('AI', 'Anguilla'),
 ('AL', 'Albania'),
 ('AM', 'Armenia'),
 ('AO', 'Angola'),
 ('AR', 'Argentina'),
 ('AS', 'American Samoa')]

## Location
---

In [14]:
locations = df[['Location', 'country']].merge(countries_codes, on='country', how='left')
# keep only unique locations
locations = locations.drop_duplicates(subset=['Location'])
# generate an id for each location
locations['Location_id'] = locations['Location'].astype('category').cat.codes
locations = locations[['Location_id', 'Location', 'country_code']]
locations = [tuple(row) for row in locations.values.tolist()]
# insert data into the table
insert_data('Location', locations)
cursor.execute('SELECT * FROM Location').fetchmany(10)

[(0, 'APAC, EMEA, US / Canada', 'AD'),
 (1, 'Aberdeen, Scotland, United Kingdom', 'DO'),
 (2, 'Aberdeen, Scotland, United Kingdom - Remote', 'MO'),
 (3, "Aix-en-Provence, Provence-Alpes-Côte d'Azur, France", 'FR'),
 (4, 'Alberta, Canada - Remote', 'MO'),
 (5, 'Alexandria, VA', 'VA'),
 (6, 'All Cities, Spain', 'AI'),
 (7, 'Amman, Amman Governorate, Jordan', 'JO'),
 (8, 'Amphoe Si Maha Phot, Thailand', 'AI'),
 (9, 'Amsterdam', 'AM')]

## Job_posts
---

In [15]:
# job posts
locations_df = df[['Location', 'country']].merge(countries_codes, on='country', how='left')
# keep only unique locations_df
locations_df = locations_df.drop_duplicates(subset=['Location'])

# generate an id for each location
locations_df['Location_id'] = locations_df['Location'].astype('category').cat.codes
locations_df = locations_df[['Location_id', 'Location', 'country_code']]
job_posts = df[['Company', 'Job Title', 'Location', 'Job Type', 'Experience level', 'Salary_in_1000_USD', 'Negociable', 'Asterisk', 'Job_category']].merge(companies_df, on='Company', how='left')
job_posts = job_posts.merge(locations_df, on='Location', how='left')
job_posts = job_posts[['Company_id', 'Job Title', 'Location_id', 'Job Type', 'Experience level', 'Salary_in_1000_USD', 'Negociable', 'Asterisk', 'Job_category']]

# change job categories values to match the enum values
job_posts['Job_category'] = job_posts['Job_category'].apply(lambda x: 'Data_Science' if x == 'Data Science' else x)
job_posts['Job_category'] = job_posts['Job_category'].apply(lambda x: 'Artificial_Intelligence' if x == 'Artificial Intelligence' else x)
job_posts['Job_category'] = job_posts['Job_category'].apply(lambda x: 'Big_Data' if x == 'Big Data' else x)
job_posts['Job_category'] = job_posts['Job_category'].apply(lambda x: 'Other' if x == 'Other' else x)

# change job types values to match the enum values
job_posts['Job Type'] = job_posts['Job Type'].apply(lambda x: 'Full_Time' if x == 'Full Time' else x)
job_posts['Job Type'] = job_posts['Job Type'].apply(lambda x: 'Part_Time' if x == 'Part Time' else x)
job_posts['Job Type'] = job_posts['Job Type'].apply(lambda x: 'Internship' if x == 'Internship' else x)

# change types to str in negociable and asterisk columns
job_posts['Negociable'] = job_posts['Negociable'].astype(int)
job_posts['Asterisk'] = job_posts['Asterisk'].astype(int)

# generate an id for each job post using len
job_posts['Id'] = [i for i in range(len(job_posts))]

job_posts = job_posts[['Id', 'Company_id', 'Location_id', 'Job Type', 'Experience level', 'Salary_in_1000_USD', 'Negociable', 'Asterisk', 'Job_category']]

# replace job_posts['Salary_in_1000_USD'] == 'Negociable' with 0
job_posts['Salary_in_1000_USD'] = job_posts['Salary_in_1000_USD'].apply(lambda x: 0 if x == 'Negociable' else x)

# as a list of tuples
job_posts = [tuple(row) for row in job_posts.values.tolist()]

# insert data into the table
insert_data('Job_posts', job_posts)

cursor.execute('SELECT * FROM Job_posts').fetchmany(5)

[(0, 810, 798, 'Full_Time', 'Entry-level', 48.0, True, True, 'Data_Science'),
 (1, 674, 288, 'Full_Time', 'Entry-level', 48.0, True, True, 'Data_Science'),
 (2, 255, 925, 'Full_Time', 'Not-Specified', 90.0, True, True, 'Artificial_Intelligence'),
 (3, 159, 627, 'Full_Time', 'Entry-level', 48.0, True, True, 'Data_Science'),
 (4, 754, 26, 'Full_Time', 'Mid-level', 108.0, True, False, 'Big_Data')]

## facilities_of_job
---

In [16]:
df['Id'] = [i for i in range(len(df))]
# get the unique values of the list
facilities = list(set([j for i in df['Facilities'].values for j in i]))
# generate an id for each facility
facilities_df = pd.DataFrame({'Facility': facilities})
facilities_df['Facility_id'] = facilities_df['Facility'].astype('category').cat.codes
facilities_df = facilities_df[['Facility_id', 'Facility']]
facilities_df = facilities_df[1:]
# for each job post get the facilities and the job id
facilities_of_the_job = df[['Facilities', 'Id']].explode('Facilities')
# drop empty facilities
facilities_of_the_job = facilities_of_the_job[facilities_of_the_job['Facilities'] != '']
facilities_of_the_job = facilities_of_the_job.merge(facilities_df, left_on='Facilities', right_on='Facility', how='left')
facilities_of_the_job = facilities_of_the_job[['Id', 'Facility_id']]
facilities_of_the_job = [tuple(row) for row in facilities_of_the_job.values.tolist()]
# insert data into the table
insert_data('Facilities_of_the_job', facilities_of_the_job)
cursor.execute('SELECT * FROM Facilities_of_the_job').fetchmany(10)

[(0, 21),
 (1, 21),
 (2, 2),
 (3, 21),
 (4, 33),
 (4, 12),
 (4, 23),
 (4, 11),
 (5, 11),
 (5, 4)]

## Requierements of job
---

In [17]:
# get the unique values of the list
job_requirements = list(set([j for i in df['Requirment of the company '].values for j in i]))
# generate an id for each job requirement
job_requirements_df = pd.DataFrame({'Job_requirement': job_requirements})
job_requirements_df['Job_requirement_id'] = job_requirements_df['Job_requirement'].astype('category').cat.codes
job_requirements_df = job_requirements_df[['Job_requirement_id', 'Job_requirement']]
job_requirements_df = job_requirements_df[1:]
# for each job post get the job requirements and the job id
job_requirements_of_the_job = df[['Requirment of the company ', 'Id']].explode('Requirment of the company ')
# drop empty job requirements
job_requirements_of_the_job = job_requirements_of_the_job[job_requirements_of_the_job['Requirment of the company '] != '']
job_requirements_of_the_job = job_requirements_of_the_job.merge(job_requirements_df, left_on='Requirment of the company ', right_on='Job_requirement', how='left')
job_requirements_of_the_job = job_requirements_of_the_job[['Id', 'Job_requirement_id']]
job_requirements_of_the_job = [tuple(row) for row in job_requirements_of_the_job.values.tolist()]
# insert data into the table
insert_data('Requirment_of_the_job', job_requirements_of_the_job)
cursor.execute('SELECT * FROM Requirment_of_the_job').fetchmany(10)

[(0, 136),
 (0, 63),
 (0, 88),
 (0, 267),
 (0, 252),
 (0, 193),
 (1, 16),
 (1, 123),
 (1, 86),
 (1, 262)]

## Job_Titles
---

In [18]:
# group Job Title in 1 list then get the unique values
df['Job Title'] = df['Job Title'].apply(lambda x: x.split(','))
df['Job Title'] = df['Job Title'].apply(lambda x: list(set(x)))

# get the unique values of the list
job_titles = list(set([j for i in df['Job Title'].values for j in i]))
# generate an id for each job title
job_titles_df = pd.DataFrame({'Job_title': job_titles})
job_titles_df['Job_title_id'] = job_titles_df['Job_title'].astype('category').cat.codes
job_titles_df = job_titles_df[['Job_title_id', 'Job_title']]
if '' in job_titles_df['Job_title'].values:
    job_titles_df = job_titles_df.drop(job_titles_df[job_titles_df['Job_title'] == ''].index)
job_titles_df = [tuple(row) for row in job_titles_df.values.tolist()]
# insert data into the table
insert_data('Job_Titles', job_titles_df)
cursor.execute('SELECT * FROM Job_Titles').fetchmany(10)

[(0, ' (Permanent Remote)'),
 (1, ' .NET CORE'),
 (2, ' 12 months (renewable)'),
 (3, ' AI'),
 (4, ' AI & Machine Learning'),
 (5, ' AI (Remote)'),
 (6, ' AI / Core Optimization'),
 (7, ' AI Platform'),
 (8, ' AI Platform and Solutions'),
 (9, ' AI Tools')]

## Job_Title_of_job_posts
---

In [19]:
# get the unique values of the list
job_titles = list(set(set([j for i in df['Job Title'].values for j in i])))
# generate an id for each job title
job_titles_df = pd.DataFrame({'Job_title': job_titles})
job_titles_df['Job_title_id'] = job_titles_df['Job_title'].astype('category').cat.codes
job_titles_df = job_titles_df[['Job_title_id', 'Job_title']]
if '' in job_titles_df['Job_title'].values:
    job_titles_df = job_titles_df.drop(job_titles_df[job_titles_df['Job_title'] == ''].index)
# for each job post get the job titles and the job id
job_titles_of_the_job = df[['Job Title', 'Id']].explode('Job Title')
# drop empty job titles
job_titles_of_the_job = job_titles_of_the_job[job_titles_of_the_job['Job Title'] != '']
job_titles_of_the_job = job_titles_of_the_job.merge(job_titles_df, left_on='Job Title', right_on='Job_title', how='left')
job_titles_of_the_job = job_titles_of_the_job[['Id', 'Job_title_id']]
job_titles_of_the_job = [tuple(row) for row in job_titles_of_the_job.values.tolist()]
# insert data into the table
insert_data('Job_titles_of_the_job', job_titles_of_the_job)
cursor.execute('SELECT * FROM Job_titles_of_the_job').fetchmany(10)

[(0, 675),
 (1, 499),
 (2, 1429),
 (3, 532),
 (4, 950),
 (5, 2121),
 (6, 2276),
 (7, 1328),
 (8, 2232),
 (9, 1562)]

## Closing connection
---

In [20]:
connection.close()